In [4]:
test_types = [[0], [1], [0, 1]]

number_of_sides = 2
for test_type_index, test_types in enumerate(test_types):
    trial_index = 0
    inputs_to_keep = test_types + [tt + number_of_sides for tt in test_types]
    print(inputs_to_keep)

print(inputs_to_keep)

[0, 2]
[1, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
